# Báo cáo Project
Lớp TTNT-162297, Nhóm G31

## 1. Thông tin chung

### Thành viên
- Phạm Hoàng Thái Dương 20235061
- Đinh Thành Đạt 202416157
- Vũ Trường Giang 20235319

### Lịch thực hiện
- W02: Đăng ký nhóm 
- W03: Đề xuất project (28/9)
- W08: Báo cáo tiến độ giữa kỳ (1/11)
- W15: Hoàn thành và gửi báo cáo project (20/12)
- W16-18: Trình bày project, Q&A

## 2. Đề xuất project (W3)

### Bài toán
Tạo AI chơi cờ vua

### Phương pháp
Sử dụng các thuật toán tìm kiếm có đối thủ như MINMAX, alpha beta prunning, kết hợp tri thức cụ thể

### Phân công
- Đạt: làm phần biểu diễn bàn cờ và thiết kế giao diện trò chơi
- Giang: thiết kế thuật toán tìm nước đi dựa trên giải thuật MINMAX và alpha beta prunning, so sánh hiệu quả 2 thuật toán
- Dương: sử dụng các tri thức cụ thể để hạn chế không gian tìm kiếm và đánh giá thế cờ hiệu quả hơn

## 3. Tiến độ giữa kỳ (W8)

### Chương trình
Hướng dẫn chạy chương trình:
1. Cài dependencies
2. Kiểm tra files và path
3. Chạy chương trình

In [ ]:
# 1. Cài dependencies
import sys
import subprocess

packages = ["pygame", "python-chess"]
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", *packages])
    print("Installed:", packages)
except subprocess.CalledProcessError as e:
    print("Failed to install packages. You may need to install them manually in your environment:", e)

In [ ]:
# 2. Kiểm tra các file cần thiết
import os, sys
cwd = os.getcwd()
print("Current working directory:", cwd)

expected = ["chess_game.py", "best_move.py", "img"]
for name in expected:
    exists = os.path.exists(os.path.join(cwd, name))
    print(f"{name}: {'FOUND' if exists else 'MISSING'}")

# Quick import test for best_move and chess libraries
try:
    import chess
    import best_move
    print("Imported chess and best_move successfully.")
except Exception as e:
    print("Import test failed:", e)

In [ ]:
# 3. Chạy chess_game.py trong một tiến trình riêng
import subprocess, sys, os

script = os.path.join(os.getcwd(), "chess_game.py")
if not os.path.exists(script):
    raise FileNotFoundError(f"{script} not found. Make sure chess_game.py is in the same folder as the notebook.")

# On Windows, use sys.executable to ensure same Python interpreter is used.
proc = subprocess.Popen([sys.executable, script], cwd=os.getcwd())
print("Started chess_game.py (PID {})".format(proc.pid))
print("To stop the game, close the pygame window. The notebook kernel remains available.")

In [ ]:
# 3*. Tùy chọn: chạy chess_game.py trực tiếp trong kernel
# cảnh báo: điều này sẽ chặn kernel của notebook trong khi cửa sổ pygame đang mở.

try:
    from chess_game import ChessGame
    print("ChessGame imported. To run it directly in the kernel, uncomment the next lines.")
    # game = ChessGame()
    # game.run()  # This will block the notebook kernel until the game window is closed.
except Exception as e:
    print("Could not import ChessGame (you can still run chess_game.py via subprocess):", e)

### Phân tích kết quả, vấn đề gặp phải

### Về mặt kết quả: 
Từ những thuật toán trên, chương trình AI chơi cờ của nhóm em đã biết cách chiếu hết đối phương và tránh bị chiếu hết, biết nắm bắt cơ hội ăn hơn quân của đối phương và tránh để bị ăn hơn quân, kiểm soát nhiều ô vuông trên bàn cờ hơn, và giữ cho quân vua của mình an toàn. Ước tính elo hiện tại của AI ở mức 1000.

### Về mặt phương pháp:

#### Thuật toán minmax trong đoạn code của em được dùng để tìm nước đi tốt nhất trên bàn cờ chess.Board:
    - Tối đa hóa điểm số với người chơi đang đến lượt
    - Tối thiểu hóa điểm số với đối thủ

#### Tổng quan cách hoạt động của file best_move:

##### 1. Hàm get_ai_move(board) là nơi bắt đầu khi cần AI đưa ra nước đi. 
Nó kiểm tra các nước đi hợp lệ (legal_moves), nếu không có thì trả None. Sau đó, gọi get_best_move_minimax() với độ sâu tìm kiếm mặc định là 3 (có thể điều chỉnh).

##### 2. Tìm nước đi tốt nhất: get_best_move_minimax(board, depth=3) duyệt qua tất cả nước đi hợp lệ từ vị trí hiện tại:

Lưu root_color = board.turn (màu của người chơi gốc, True cho trắng, False cho đen) để đánh giá luôn từ góc nhìn của người chơi này (tối đa hóa lợi ích cho họ).
Khởi tạo best_move là nước đi đầu tiên (dự phòng), best_score = -inf.

Với mỗi nước đi (move):

- board.push(move): Thực hiện nước đi trên bàn cờ.
- Gọi minimax(board, depth-1, False, -inf, inf, root_color): Bắt đầu đệ quy từ lượt đối thủ (minimizing, tức là maximizing: False), tìm score tối ưu sau nước đi này.
- board.pop(): Hoàn tác nước đi, quay về trạng thái ban đầu.
- Nếu score > best_score, cập nhật best_score và best_move.

Trả về best_move tốt nhất (nước đi cho điểm số cao nhất từ góc nhìn người chơi gốc).

Thuật toán cốt lõi: Minimax với Alpha-Beta: Hàm minimax(board, depth, maximizing, alpha, beta, root_color) tính toán score tối ưu cho vị trí hiện tại:

Điều kiện dừng: Nếu depth == 0 hoặc trò chơi kết thúc (board.is_game_over()), gọi evaluate_position(board, root_color) để đánh giá vị trí tĩnh (static evaluation).

##### 3. Hàm minimax(board: chess.Board, depth: int, maximizing: bool, alpha: float, beta: float, root_color: bool) -> float:

Các tham số:
board: chess.Board, bàn cờ hiện tại
depth: độ sâu tính toán
maximizing: True nếu gọi hàm max, false nếu gọi hàm min
alpha: cận dưới của hàm max, liên tục được cập nhật
beta: cận trên của hàm min, liên tục được cập nhật


##### Lượt tối đa hóa (maximizing = True): tối đa hóa số điểm cho bên đến lượt hiện tại

Khởi tạo max_eval = -inf.
Duyệt từng nước đi hợp lệ:

Push move, đệ quy gọi minimax với depth-1, maximizing: False (minimizing), pop move.

Cập nhật max_eval = max(max_eval, eval_score).

Cập nhật alpha = max(alpha, eval_score) (giá trị tốt nhất mà Max có thể đảm bảo).
Nếu beta <= alpha, break: Cắt tỉa nhánh

Trả về max_eval.

##### Lượt tối thiểu hóa (maximizing = False): Lượt đối thủ (tối thiểu hóa số điểm cho bên đến lượt hiện tại).

Tương tự, khởi tạo min_eval = inf.
Duyệt từng nước đi hợp lệ:

Push move, đệ quy gọi minimax với depth-1, maximizing: True (minimizing), pop move.

Cập nhật min_eval = min(min_eval, eval_score), beta = min(beta, eval_score).
Nếu beta <= alpha, break: Cắt tỉa nhánh

##### 4. Đánh giá thế cờ: evaluate_position(board, root_color): Tính score tĩnh cho bàn cờ từ góc nhìn root_color (dương nếu lợi cho họ).

Trường hợp kết thúc:

Nếu checkmate: Nếu lượt hiện tại là root_color (họ bị chiếu hết), trả -inf; ngược lại inf.
Nếu stalemate, insufficient material, hoặc lặp vị trí 3 lần: Trả 0 (hòa).

Score cơ bản: Duyệt qua tất cả ô vuông (chess.SQUARES):

Lấy piece tại ô, gán giá trị (pawn=1, knight = 3, bishop=3, rook=5, queen=9, king=0).

Nếu piece thuộc root_color, cộng giá trị; ngược lại trừ.

Yếu tố bổ sung:

Mobility (tính di động): 
Sao chép bàn cờ tạm (b1 = board.copy()), đặt lượt cho root_color và tính số legal moves (mobility_root). Tương tự cho đối thủ (mobility_opp). Thêm 0.05 * (mobility_root - mobility_opp) vào score (ưu tiên vị trí có nhiều lựa chọn hơn).
Square control (kiểm soát ô): Gọi count_square_control(), thêm 0.01 * kết quả (dương nếu root_color kiểm soát nhiều ô hơn).

King safety (an toàn vua): 
Gọi king_safety(), thêm 0.1 * kết quả (dương nếu vua của root_color được che chắn tốt hơn).

Tổng score là sự kết hợp các yếu tố này, giúp AI ưu tiên vị trí cân bằng (không chỉ đếm quân).

Hàm phụ trợ:

count_square_control(board, root_color): Duyệt từng ô có piece, tính số ô nó tấn công (board.attacks(square)). Nếu piece thuộc root_color, cộng số điểm tương ứng với số ô tấn công; ngược lại trừ. Trả về một giá trị controlled (controlled > 0 lợi cho root_color, controlled < 0 lợi cho đối thủ).

king_safety(board, root_color): Với từng màu (white/black):
Tìm vị trí vua (board.king(color)).
Xác định hướng pawn che chắn (lên cho trắng, xuống cho đen).
Kiểm tra 3 ô phía trước vua (file -1, 0, +1), nếu có pawn đồng màu thì cộng 0.5 vào shield.
Nếu color là root_color, cộng shield vào total; ngược lại trừ.
Trả về total (dương nếu vua root_color an toàn hơn).

### Về một số vấn đề gặp phải:

1. Các đánh giá của hàm evaluate_position chưa tối ưu:

- Việc có tổng giá trị quân cờ lớn hơn đối phương chưa chắc là con đường tốt nhất để dẫn đến chiến thắng, đôi khi các kì thủ mạnh có thể thí quân có giá trị cao hơn để đổi lấy thế trận mạnh (ví dụ: thí quân xe yếu lấy quân mã mạnh)

- Việc kiểm soát nhiều ô vuông trên bàn cờ chưa chắc đã đem lại lợi thế. Thay vào đó, trong ván cờ thực tế người ta chú trọng nhiều hơn đến việc kiểm soát các ô vuông quan trọng tùy thuộc vào tình huống cụ thể (ví dụ: ô trung tâm, ô vuông gần vua đối phương)

- Mức độ an toàn của quân vua trên thực tế là một khái niệm cảm tính, khó có thể đánh giá được chỉ dựa trên hàng tốt phía trước vua.


2. Việc phải xét tất cả các nước đi mỗi lần gọi hàm minmax là chưa tối ưu và chưa cần thiết:

- Một kì thủ mạnh chỉ xem xét nhiều nhất 5 đến 6 lựa chọn nước đi trên mỗi thế cờ, và tính toán chi tiết những nước ở độ sâu khoảng 10 nước, thay vì tính toán tất cả các nước hợp lệ ở độ sâu 3.

    + Tuy nhiên, việc lựa chọn nước đi của kì thủ con người phụ thuộc rất lớn vào những kiến thức cụ thể cũng như cảm quan và kinh nghiệm của họ, điều này rất khó để mô phỏng bằng những dòng code.


Từ những khó khăn trên, em dự định cải thiện AI bằng những phương pháp sau:

    - Cải thiện hàm heuristic để đánh giá thế cờ hiệu quả hơn: 
        + Thêm đánh giá cấu trúc tốt
        + Cộng thêm điểm khi quân cờ kiểm soát những ô vuông trọng yếu
        + Trừ điểm an toàn vua nếu ô vuông xung quanh vua bị đối phương kiểm soát
        + ... 

    - Kết hợp với học sâu:
        +Thiết lập mạng nơ ron phù hợp và train AI bằng cách cho AI đánh cờ với stockfish - chương trình đánh cờ vua mạnh nhất thế giới hiện tại

## 4. Cập nhật kết quả cuối kỳ (W15)

### Chi tiết phương pháp, dữ liệu 

....

### Chương trình
...

### Phân tích, đánh giá kết quả
...

### Cập nhật phân công, khối lượng công việc
<!-- công việc của các thành viên, tỷ lệ đóng góp của các thành viên -->
...